In [4]:
# Importing libraries
import sys
import os
import re
import pyap
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

In [18]:
def convertPDFToText(path):
    """
    This function converts pdf to text

    path: path of the file to be converted
    """
    fp = open(path, 'rb')
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'ascii'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    # Create a PDF interpreter object.
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    # Process each page contained in the document.

    for page in PDFPage.get_pages(fp):
        interpreter.process_page(page)
        data =  retstr.getvalue()

    return data

In [29]:
a = convertPDFToText("resume_trail1.pdf")

In [22]:
import string

In [20]:
def preprocess(f):
    return "\n".join([el.strip() for el in f.split("\n") if len(el.strip()) > 1])

In [25]:
printable = set(string.printable)
a = filter(lambda x: x in printable, a)

In [33]:
preprocess(a)

'Managing Partner and Program Manager/Engineer\nErick B. Knezek\nEducation\n\uf0a7  B.S., Ocean Engineering, United\nStates Naval Academy, 1996\n\uf0a7  M.S., Ocean Engineering, Texas A\n& M, 1997\nYears of Experience - 18\nProfessional Engineer\nMaryland,\nQualified Navy Deep Sea Diver\nand Dive Supervisor\nRELEVANT EXPERIENCE\nMr.  Knezek  has  more  than  18  years  of  program/technical\nmanagement,  civil  and  ocean  engineering,  and  construction\nexperience.  He  has  managed  numerous  projects  over  the  past  14\nyears  while  executing  engineering  projects  worldwide  including\nmanagement of pier design, installation and repair projects.  Erick\nhas  been\nthe  planning,  program  policy\ndevelopment,\nfinancial  management,  manpower  allocation,\nlogistics,  and  technical  management  of  construction  activities  at\nfacilities worldwide.\nresponsible\nfor\nMr.  Knezek  is  a  former  US  Navy  Civil  Engineer  Corps  Officer.\nThat  along  with  his  background  a

In [32]:
re.sub(r'[^\x00-\x7f]', "", preprocess(a))

'Managing Partner and Program Manager/Engineer\nErick B. Knezek\nEducation\n  B.S., Ocean Engineering, United\nStates Naval Academy, 1996\n  M.S., Ocean Engineering, Texas A\n& M, 1997\nYears of Experience - 18\nProfessional Engineer\nMaryland,\nQualified Navy Deep Sea Diver\nand Dive Supervisor\nRELEVANT EXPERIENCE\nMr.  Knezek  has  more  than  18  years  of  program/technical\nmanagement,  civil  and  ocean  engineering,  and  construction\nexperience.  He  has  managed  numerous  projects  over  the  past  14\nyears  while  executing  engineering  projects  worldwide  including\nmanagement of pier design, installation and repair projects.  Erick\nhas  been\nthe  planning,  program  policy\ndevelopment,\nfinancial  management,  manpower  allocation,\nlogistics,  and  technical  management  of  construction  activities  at\nfacilities worldwide.\nresponsible\nfor\nMr.  Knezek  is  a  former  US  Navy  Civil  Engineer  Corps  Officer.\nThat  along  with  his  background  as  a  Profes

In [ ]:
def preprocess(document):
        """
        This function preprocess the document and removes all the noise
        
        
        """
        return re.sub(r'[^\x00-\x7f]',"","\n".join([el.strip() for el in f.split("\n") if len(el.strip()) > 1]))
        # Newlines are one element of structure in the data
        # Helps limit the context and breaks up the data as is intended in resumes - i.e., into points
        lines = [el.strip() for el in document.split("\n") if len(el) > 0]  # Splitting on the basis of newlines 
        lines = [nltk.word_tokenize(el) for el in lines]    # Tokenize the individual lines
        lines = [nltk.pos_tag(el) for el in lines]  # Tag them
        # Below approach is slightly different because it splits sentences not just on the basis of newlines, but also full stops 
        # - (barring abbreviations etc.)
        # But it fails miserably at predicting names, so currently using it only for tokenization of the whole document
        sentences = nltk.sent_tokenize(document)    # Split/Tokenize into sentences (List of strings)
        sentences = [nltk.word_tokenize(sent) for sent in sentences]    # Split/Tokenize sentences into words (List of lists of strings)
        tokens = sentences
        sentences = [nltk.pos_tag(sent) for sent in sentences]    # Tag the tokens - list of lists of tuples - each tuple is (<word>, <tag>)
        # Next 4 lines convert tokens from a list of list of strings to a list of strings; basically stitches them together
        dummy = []
        for el in tokens:
            dummy += el
        tokens = dummy
        # tokens - words extracted from the doc, lines - split only based on newlines (may have more than one sentence)
        # sentences - split on the basis of rules of grammar
        return tokens, lines, sentences

In [68]:
def Name(text):
    """
    This function gives the names from the given text
        
    text: Text from which names has to be extracted
    """
    Names = open("first_names.txt", "r").read().lower()
    # Lookup in a set is much faster
    Names = set(Names.split("\n"))
    otherNameHits = []
    nameHits = []
    name = None
    lines = [el.strip() for el in text.split("\n") if len(el) > 0]
    lines = [nltk.word_tokenize(el) for el in lines]
    tokens = lines
    lines = [nltk.pos_tag(el) for el in lines]
    grammar = r'NAME: {<NN.*><NN.*><NN.*><NN.*>*}'
    chunkParser = nltk.RegexpParser(grammar)
    all_chunked_tokens = []
    for tagged_tokens in lines:
        if len(tagged_tokens) == 0: continue # Prevent it from printing warnings
        chunked_tokens = chunkParser.parse(tagged_tokens)
        all_chunked_tokens.append(chunked_tokens)
        for subtree in chunked_tokens.subtrees():
            if subtree.label() == 'NAME':
                for ind, leaf in enumerate(subtree.leaves()):
                    if leaf[0].lower() in Names and 'NN' in leaf[1]:
                        hit = " ".join([el[0] for el in subtree.leaves()[ind:ind+3]])
                        if re.compile(r'[\d,:]').search(hit): continue
                        nameHits.append(hit)
    if len(nameHits) > 0:
        nameHits = [re.sub(r'[^a-zA-Z \-]', '', el).strip() for el in nameHits] 
        name = " ".join([el[0].upper()+el[1:].lower() for el in nameHits[0].split() if len(el)>0])
        otherNameHits = nameHits[1:]
    print(name)
    return name, otherNameHits

In [69]:
f = convertPDFToText("resume_trail1.pdf")

In [70]:
Name(f)

Erick B Knezek


('Erick B Knezek',
 ['Diver',
  'Truston Technologies Ocean',
  'Truston Technologies Waterfront',
  'Truston Technologies Marine',
  'Erick B Knezek',
  'ITA Industrial'])